In [11]:
# Read url
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/lkyin/ECS189L/main/Tweets.csv')

# Preview
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')
print('Done!')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Done!


## ***Q1***


In [ ]:
# Q1
## initialization
Q1_df = df
airline_sentiment = df['airline_sentiment']

## Store airline names into an array
airline_names = df["airline"].unique()

## Remove punctuation
Q1_df['text'] = Q1_df['text'].str.replace("!", "")
Q1_df['text'] = Q1_df['text'].str.replace(".", "")
Q1_df['text'] = Q1_df['text'].str.replace("#", "")
Q1_df['text'] = Q1_df['text'].str.replace(",", "")
Q1_df['text'] = Q1_df['text'].str.replace(":", "")
Q1_df['text'] = Q1_df['text'].str.replace(")", "")
Q1_df['text'] = Q1_df['text'].str.replace("(", "")
Q1_df['text'] = Q1_df['text'].str.replace("?", "")
Q1_df['text'] = Q1_df['text'].str.replace("-", "")
Q1_df['text'] = Q1_df['text'].str.replace("I", "")
Q1_df['text'] = Q1_df['text'].str.replace("'", "")
Q1_df['text'] = Q1_df['text'].str.replace("&", "")
Q1_df['text'] = Q1_df['text'].str.replace(";", "")
Q1_df['text'] = Q1_df['text'].str.replace("|", "")
Q1_df['text'] = Q1_df['text'].str.replace('”', "")
Q1_df['text'] = Q1_df['text'].str.replace('“', "")
Q1_df['text'] = Q1_df['text'].str.replace('``', "")
Q1_df['text'] = Q1_df['text'].str.replace('"', "")

# Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
for index, value in Q1_df['text'].items():
  # Lower Case
  value = ps.stem(value.lower())
  Q1_df['text'].iloc[index] = value

## Remove @airline
for values in airline_names:
  # Add @ sign before airline names
  values = "@" + values
  # Remove empty space
  values = values.replace(" ","")
  if (values == "@American"):
    values = "@AmericanAir"
  elif (values == "@Southwest"):
    values = "@SouthwestAir"
  # Replace
  Q1_df['text'] = Q1_df['text'].str.replace(values, "")

Q1_df['text'] = Q1_df['text'].str.replace("@USairways", "")
Q1_df['text'] = Q1_df['text'].str.replace("@UsAirways", "")
Q1_df['text'] = Q1_df['text'].str.replace("@JetBlue", "")

## Remove all lower case @airline
Q1_df['text'] = Q1_df['text'].str.replace('@united', "")
Q1_df['text'] = Q1_df['text'].str.replace('@jetblue', "")
Q1_df['text'] = Q1_df['text'].str.replace('@southwestair', "")
Q1_df['text'] = Q1_df['text'].str.replace('@americanair', "")
Q1_df['text'] = Q1_df['text'].str.replace('@usairways', "")
Q1_df['text'] = Q1_df['text'].str.replace('@virginamerica', "")
Q1_df['text'] = Q1_df['text'].str.replace("@", "")

# Preview
print("\n")
Q1_df['text'].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
## Split into three sentiment groups: positive, neutral, negative
Q1_positive = df[airline_sentiment == 'positive']
Q1_neutral = df[airline_sentiment == 'neutral']
Q1_negative = df[airline_sentiment == 'negative']

## Positive sentiments
## Remove stopwords
wordlist = []
for index, value in Q1_positive['text'].items():
  stop_words = set(stopwords.words('english'))
  ## Tokenizing
  word_tokens = word_tokenize(value)
  for w in word_tokens:
     if not w in stop_words:
       wordlist.append(w)

## Report
from collections import Counter
print("Top 10 words used in positive sentiment group are:")
print(Counter(wordlist).most_common(10))

In [ ]:
## Neutral sentiments
## Remove stopwords
wordlist = []
for index, value in Q1_neutral['text'].items():
  stop_words = set(stopwords.words('english'))
  ## Tokenizing
  word_tokens = word_tokenize(value)
  for w in word_tokens:
     if not w in stop_words:
       wordlist.append(w)

## Report
print("Top 10 words used in neutral sentiment group are:")
print(Counter(wordlist).most_common(10))

In [ ]:
## Negative sentiments
## Remove stopwords
wordlist = []
for index, value in Q1_negative['text'].items():
  stop_words = set(stopwords.words('english'))
  ## Tokenizing
  word_tokens = word_tokenize(value)
  for w in word_tokens:
     if w not in stop_words:
       wordlist.append(w)

## Report
print("Top 10 words used in negative sentiment group are:")
print(Counter(wordlist).most_common(10))

## ***Q2***

In [ ]:
# Q2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import numpy as np

## Predictors and output
X = Q1_df['text']
Y = Q1_df['airline_sentiment']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

## Split dataset into 80% for training and 20% for testing
## stratify parameter ensures each airline is represented in training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state = 23)

## Train
model = MultinomialNB()
model.fit(X_train, y_train)
print('Done Training')

## Predict
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred, output_dict = True)
print(pd.DataFrame(report).transpose())

## Cross Validation for not overfitting
scores = cross_val_score(model, X, Y, cv=10)

In [ ]:
## Report
from sklearn.metrics import accuracy_score, precision_score, recall_score
print('Accuracy score for this model is: %0.3f' % accuracy_score(y_test, y_pred))
print("Accuracy score for Cross Validation: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## ***Q3***


In [ ]:
# Q3
# Assuming fraction over number of sentiments specific to each airline
airline_names = df["airline"].unique()

# Frequency of positive, neutral, and negative sentiment for each airline
neg = {"Virgin America": 0 ,"United" : 0, "Southwest":0 , "Delta": 0, "US Airways" : 0, "American" : 0}
neutral = {"Virgin America": 0 ,"United" : 0, "Southwest":0 , "Delta": 0, "US Airways" : 0, "American" : 0}
pos = {"Virgin America": 0 ,"United" : 0, "Southwest":0 , "Delta": 0, "US Airways" : 0, "American" : 0}

# Fraction of positives and negatives for each airline
neg_frac = {"Virgin America": 0 ,"United" : 0, "Southwest":0 , "Delta": 0, "US Airways" : 0, "American" : 0}
pos_frac = {"Virgin America": 0 ,"United" : 0, "Southwest":0 , "Delta": 0, "US Airways" : 0, "American" : 0}

for airline in airline_names:
  # Extract airline series
  cur_airline = df[df['airline'] == airline]

  # Find the frequency for positive, negative, and neutral
  counts = cur_airline['airline_sentiment'].value_counts()

  # Append each value to our frequency dictionary
  neg[airline] = (counts[0])
  neutral[airline] = (counts[1])
  pos[airline] = (counts[2])

  # Calculate total number of sentiments for each airline and calculate fraction
  total = counts[0] + counts[1] + counts[2]
  neg_frac[airline] = counts[0] / total
  pos_frac[airline] = counts[2] / total

# Number of positives and negatives for each airline
print("Negative dict:", neg)
print("Positive dict:", pos)
print("Neutral dict:", neutral)

# Fraction of positives and negatives for each airline
print("\nNegative fraction dict:", neg_frac)
print("Positive fraction dict:", pos_frac)

# Sort the dictionary
print("\nFraction with Negative sentiments in decreasing value:")
sorted_name_neg = sorted(neg_frac, key=neg_frac.get, reverse = True)
for name in sorted_name_neg:
    print(name, neg_frac[name])

print("\nFraction with positive sentiments in decreasing value:")
sorted_name_neg = sorted(pos_frac, key=pos_frac.get, reverse = True)
for name in sorted_name_neg:
    print(name, pos_frac[name])

In [ ]:
# Report
print("The top 3 airlines in terms of the fraction of negative tweets are: US Airways, American, United")
print("The top 3 airlines in terms of the fraction of positive tweets are: Virgin America, Delta, Southwest")